In [ ]:
'''
Step 1. Upload the file and read
'''

data = open('ko_kaist-ud.txt', 'r', encoding='utf-8') # encoding='utf-8' : Korean data set

lines = data.readlines() #read by lines
data.close()

len(lines) #31468
lines[:20] #sample 20


['1\t내\t내\tADJ\tmma\t_\t2\tamod\t_\t_\n',
 '2\t고향은\t고향+은\tNOUN\tncn+jxt\t_\t3\tdislocated\t_\t_\n',
 '3\t서울입니다\t서울+이+ㅂ니다\tVERB\tnq+jp+ef\t_\t0\troot\t_\tSpaceAfter=No\n',
 '4\t.\t.\tPUNCT\tsf\t_\t3\tpunct\t_\t_\n',
 '1\t옛날의\t옛날+의\tNOUN\tncn+jcm\t_\t2\tnmod\t_\t_\n',
 '2\t서울의\t서울+의\tPROPN\tnq+jcm\t_\t3\tnmod\t_\t_\n',
 '3\t모습이\t모습+이\tNOUN\tncn+jcs\t_\t5\tnsubj\t_\t_\n',
 '4\t몹시\t몹시\tADV\tmag\t_\t5\tadvmod\t_\t_\n',
 '5\t그립습니다\t그립+습니다\tADJ\tpaa+ef\t_\t0\troot\t_\tSpaceAfter=No\n',
 '6\t.\t.\tPUNCT\tsf\t_\t5\tpunct\t_\t_\n',
 '1\t이른\t이르+ㄴ\tADJ\tpaa+etm\t_\t2\tamod\t_\t_\n',
 '2\t아침과\t아침+과\tCCONJ\tncn+jcj\t_\t4\tdep\t_\t_\n',
 '3\t밤\t밤\tNOUN\tncn\t_\t2\tconj\t_\t_\n',
 '4\t늦게\t늦+게\tSCONJ\tpaa+ecs\t_\t12\txcomp\t_\t_\n',
 '5\t종로\t종로\tPROPN\tnq\t_\t7\tnmod\t_\t_\n',
 '6\t보신각의\t보신각+의\tPROPN\tnq+jcm\t_\t5\tflat\t_\t_\n',
 '7\t종소리를\t종소리+를\tNOUN\tncn+jco\t_\t8\tobj\t_\t_\n',
 '8\t듣고\t듣+고\tCCONJ\tpvg+ecc\t_\t14\tacl\t_\t_\n',
 '9\t잠을\t잠+을\tNOUN\tncn+jco\t_\t10\tobj\t_\t_\n',
 '10\t자고\t자+고\tCCONJ\

In [ ]:
index = []
word = []
pos = []

for i, line in enumerate(lines):
    line = line.strip() # delete '\n'
    splits = line.split('\t') # split by '\t'

    if len(splits) > 2: # filter '\n\'
        try:
            idx = int(splits[0])
            index.append(idx)
            word.append(splits[1])
            pos.append(splits[3])

        except:
            print(i, index[-5:], word[-5:], pos[-5:])
            print(line) #check if there is an error in idx
            # or use print(set(index))

In [ ]:
print (index[:20])
print (word[:20])
print (pos[:20])

[1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
['내', '고향은', '서울입니다', '.', '옛날의', '서울의', '모습이', '몹시', '그립습니다', '.', '이른', '아침과', '밤', '늦게', '종로', '보신각의', '종소리를', '듣고', '잠을', '자고']
['ADJ', 'NOUN', 'VERB', 'PUNCT', 'NOUN', 'PROPN', 'NOUN', 'ADV', 'ADJ', 'PUNCT', 'ADJ', 'CCONJ', 'NOUN', 'SCONJ', 'PROPN', 'PROPN', 'NOUN', 'CCONJ', 'NOUN', 'CCONJ']


In [ ]:
'''
Step 2. Formating the data
- Represent the dataset as a list of dictionaries
- Represent words as dictionaries with multiple features
'''

input_data = [] #total sentences

sent = [] #make a sentence->input_data
i = 0
pos_index = 0

while (i < len(index)):
    if (int(index[i])) == pos_index + 1:
          sent.append({"word" :word[i], "pos" :pos[i]})
          pos_index += 1
          i = i +1
    else:
        if sent:
            input_data.append(sent) #to filterout empty index
        else:
            i = i +1
        pos_index = 0 #new sentence, make the number of sentence to default number
        sent = []
        if i == (len(index)-1):
            break
    
    if i % 10000 == 0:
        print (i, len(index)) #check an error

10000 25278
20000 25278


In [ ]:
len(input_data) #the number of total sentences

2065

In [ ]:
full_data = input_data
print (full_data[1][:10])

[{'word': '옛날의', 'pos': 'NOUN'}, {'word': '서울의', 'pos': 'PROPN'}, {'word': '모습이', 'pos': 'NOUN'}, {'word': '몹시', 'pos': 'ADV'}, {'word': '그립습니다', 'pos': 'ADJ'}, {'word': '.', 'pos': 'PUNCT'}]


In [ ]:
'''
Step 3. Format the data for scikit-learn and extract the features
'''


def simple_features(input_sent,idx,token): #takes a sentence as input (with word and tag specified), outputs a more feature-rich version
	features = {}
	#features["word"] = token["word"]
	if idx == 0:
		features["prev_pos"] = "<start>" #no previous pos

	elif idx == 1:
		features["prev_pos"] = input_sent[idx-1]["pos"] #previos pos_tag

	else:
		features["prev_pos"] = input_sent[idx-1]["pos"] #

	return(features)

In [ ]:
def feature_extractor(input_data): #takes list [sents] of lists [tokens] of dictionaries [token_features], outputs a flat list of dicts [features]
	feature_list = [] #flast list of token dictionaries
	for sent in input_data: #iterate through sentences
		for idx, token in enumerate(sent): #iterate through tokens
			feature_list.append(simple_features(sent,idx,token)) #use simple_features function to add features
	return(feature_list)

def extract_pos(input_data):
	pos_list = []
	for sent in input_data:
		for token in sent:
			pos_list.append(token["pos"])
	return(pos_list)

def extract_words(input_data):
	word_list = []
	for sent in input_data:
		for token in sent:
			word_list.append(token["word"])
	return(word_list)

flat_words = extract_words(full_data)
flat_pos = extract_pos(full_data)
flat_features = feature_extractor(full_data)

In [ ]:
for idx, x in enumerate(flat_words[:10]):
  print(x, flat_pos[idx], flat_features[idx])

내 ADJ {'prev_pos': '<start>'}
고향은 NOUN {'prev_pos': 'ADJ'}
서울입니다 VERB {'prev_pos': 'NOUN'}
. PUNCT {'prev_pos': 'VERB'}
옛날의 NOUN {'prev_pos': '<start>'}
서울의 PROPN {'prev_pos': 'NOUN'}
모습이 NOUN {'prev_pos': 'PROPN'}
몹시 ADV {'prev_pos': 'NOUN'}
그립습니다 ADJ {'prev_pos': 'ADV'}
. PUNCT {'prev_pos': 'ADJ'}


In [ ]:
print (len(flat_words) * .67) #16928

#training data
train_words = flat_words[:16928]
train_pos = flat_pos[:16928]
train_features = flat_features[:16928]

#test data
test_words = flat_words[16928:]
test_pos = flat_pos[16928:]
test_features = flat_features[16928:]

16927.55


In [ ]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse = True)

#transform categorical variables to vectors

#we use .fit_transform() to create the vectors
train_features_vec = vec.fit_transform(train_features) #vectorize sample of features

#and apply previously made vectors using .transform()
test_features_vec = vec.transform(test_features)

#create our own POS conversion dictionary
def pos_cats(pos_list):
	cat_d = {}
	for idx, x in enumerate(list(set(pos_list))):
		cat_d[x] = idx
	return(cat_d)

pos_d = pos_cats(flat_pos)
#print(pos_d)

def convert_pos(pos_list,pos_d):
	converted = []
	for x in pos_list:
		converted.append(pos_d[x])
	return(converted)

train_pos_num = convert_pos(train_pos,pos_d)

In [ ]:
train_pos_num[:10]

[8, 9, 16, 6, 9, 3, 9, 11, 8, 6]

In [ ]:
from sklearn import tree #import decision tree module

clf = tree.DecisionTreeClassifier() #create classifier
clf = clf.fit(train_features_vec,train_pos_num) #train model (features, pos tags)

pred1 = clf.predict(test_features_vec) #apply model to new data

print(pred1[:10]) #print first ten items in pred1

[16  9  9  9  9  9  9  6  9  9]


In [ ]:
rev_pos_d = {value : key for (key, value) in pos_d.items()}

def extract_pred_pos(pred_array,rev_d):
	predicted = []
	for x in pred_array:
		predicted.append(rev_d[x])
	return(predicted)

pred1_pos = extract_pred_pos(pred1,rev_pos_d)
pred1_pos[:10]

['VERB',
 'NOUN',
 'NOUN',
 'NOUN',
 'NOUN',
 'NOUN',
 'NOUN',
 'PUNCT',
 'NOUN',
 'NOUN']

In [ ]:
def pred_accuracy(pred,gold):
	c = 0
	f = 0

	for idx, x in enumerate(pred):
		#print(x,gold[idx]["pos"])
		if x == gold[idx]:
			c+=1
		else:
			f+=1
	return(c/(c+f))

pred_accuracy(pred1_pos,test_pos) #0.3284 ...???

0.32841549718124025

In [ ]:
def simple_features2(input_sent,idx,token): #takes a sentence as input (with word and tag specified), outputs a more feature-rich version
	features = {}
	features["word"] = token["word"]
	if idx == 0:
		features["prev_pos"] = "<start>" #no previous pos

	elif idx == 1:
		features["prev_pos"] = input_sent[idx-1]["pos"] #previos pos_tag

	else:
		features["prev_pos"] = input_sent[idx-1]["pos"] #

	features["suffix_tg"] = token["word"][-3:] #get last three characters

	return(features)

def feature_extractor2(input_data): #takes list [sents] of lists [tokens] of dictionaries [token_features], outputs a flat list of dicts [features]
	feature_list = [] #flast list of token dictionaries
	for sent in input_data: #iterate through sentences
		for idx, token in enumerate(sent): #iterate through tokens
			feature_list.append(simple_features2(sent,idx,token)) #use simple_features function to add features
	return(feature_list)

flat_features2 = feature_extractor2(full_data)

print(flat_features2[0])

train_features2 = flat_features2[:16928]
test_features2 = flat_features2[16928:]

vec2 = DictVectorizer(sparse = True)

#transform categorical variables to vectors

#we use .fit_transform() to create the vectors
train_features2_vec = vec2.fit_transform(train_features2) #vectorize sample of features

#and apply previously made vectors using .transform()
test_features2_vec = vec2.transform(test_features2)

clf2 = tree.DecisionTreeClassifier()
clf2 = clf2.fit(train_features2_vec,train_pos_num)

pred2 = clf2.predict(test_features2_vec)
pred2_pos = extract_pred_pos(pred2,rev_pos_d)
#check accuracy
print (pred_accuracy(pred2_pos,test_pos)) #0.6269

{'word': '내', 'prev_pos': '<start>', 'suffix_tg': '내'}
0.6269641357802567


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
